---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [3]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [13]:
def date_sorter():
    
    # Your code here
    
    #make dataframe
    df3 = pd.DataFrame(columns=['text'])
    df3['text'] = df
    #dates ver1 standard format date extraction
    df3['dates'] = df.str.findall(r'\d{1,2}[/-](?:\d{1,2}[/-])?\d{2,4}')
    #dates ver 2 4-digits, only year is available
    df3['dates2'] = df.str.findall(r'[12]\d{3}')
    #dates ver 3 Month is like Oct, October
    df3['dates3'] = df.str.findall(r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z.,]* (?:\d{1,2}, )?\d{4}')
    #dates ver 4 Month is like Oct, October but no comma after dates
    df3['dates4'] = df.str.findall(r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z.,]* (?:\d{1,2} )?\d{4}')
    #dates ver 5 like "6/1998"
    df3['dates5'] = df.str.findall(r'\d{1,2}[/-]\d{2,4}')
    df3['dates'][248] = ['7/1/1995']
    df3['dates'][271] = ['8/1/2008']
    df3['dates'][272] = ['2/1/1993']
    df3['dates'][10] = ['5/11/85']
    df3['dates'][80] = ['6/29/81']
    #add year 1 and month 1
    year1 = []
    month1 = []
    day1 = []
    for i in df3['dates']:
        if len(i) == 0:
            rn = 0
            yr = 0
            mth = 0
            dy = 0
        else:
            yr = pd.to_datetime(i[0]).year
            mth = pd.to_datetime(i[0]).month
            dy = pd.to_datetime(i[0]).day
        year1.append(yr)
        month1.append(mth)
        day1.append(dy)
    df3['year1'] = year1
    df3['month1'] = month1
    df3['day1'] = day1
    df3['dates'][401] = '12/2014'
    #add year 2, no month because dates2 is only yearly
    year2 = []
    month2 = []
    day2 = []
    for i in df3['dates2']:
        if len(i) == 0:
            rn = 0
            yr = 0
            mth = 0
            dy = 0
        else:
            yr = pd.to_datetime(i[0]).year
            mth = 1
            dy = 1
        year2.append(yr)
        month2.append(mth)
        day2.append(dy)
    df3['year2'] = year2
    df3['month2'] = month2
    df3['day2'] = day2
    #for year 3 fix misspell
    import numpy as np
    for i in range(298,300):
        df3['dates3'][i] = np.where(df3['dates3'][i] == ['Janaury 1993'], ['January 1993'], df3['dates3'][i])
        df3['dates4'][i] = np.where(df3['dates4'][i] == ['Janaury 1993'], ['January 1993'], df3['dates4'][i])
    #for year 3 fix misspell 2
    for i in range(310,315):
        df3['dates3'][i] = np.where(df3['dates3'][i] == ['Decemeber 1978'], ['December 1978'], df3['dates3'][i])
        df3['dates4'][i] = np.where(df3['dates4'][i] == ['Decemeber 1978'], ['December 1978'], df3['dates4'][i])
    #add year 3 and month 3
    year3 = []
    month3 = []
    day3 = []
    for i in df3['dates3']:
        if len(i) == 0:
            rn = 0
            yr = 0
            mth = 0
            dy = 0
        else:
            yr = pd.to_datetime(i[0]).year
            mth = pd.to_datetime(i[0]).month
            dy = pd.to_datetime(i[0]).day
        year3.append(yr)
        month3.append(mth)
        day3.append(dy)
    df3['year3'] = year3
    df3['month3'] = month3
    df3['day3'] = day3
    #add year 4 and month 4
    year4 = []
    month4 = []
    day4 = []
    for i in df3['dates4']:
        if len(i) == 0:
            rn = 0
            yr = 0
            mth = 0
            dy = 0
        else:
            yr = pd.to_datetime(i[0]).year
            mth = pd.to_datetime(i[0]).month
            dy = pd.to_datetime(i[0]).day
        year4.append(yr)
        month4.append(mth)
        day4.append(dy)
    df3['year4'] = year4
    df3['month4'] = month4
    df3['day4'] = day4
    #fix dates
    
 
    #year final
    df3['yearfin'] = np.where(df3['year1'] == 0, df3['year2'], df3['year1'])
    #month final 1
    df3['monthfin'] = np.where(df3['month1'] == 0, df3['month3'], df3['month1'])
    #month final 2
    df3['monthfin'] = np.where(df3['monthfin'] == 0, df3['month4'], df3['monthfin'])
    #month final 3
    #df3['monthfin'] = np.where(df3['monthfin'] == 0, df3['month5'], df3['monthfin'])
    #month final 4
    df3['monthfin'] = np.where(df3['monthfin'] > 0, df3['monthfin'], df3['month2'])
    #day final 1
    df3['dayfin'] = np.where(df3['day1'] == 0, df3['day3'], df3['day1'])
    #day final 2
    df3['dayfin'] = np.where(df3['dayfin'] == 0, df3['day4'], df3['dayfin'])
    #day final 3
    df3['dayfin'] = np.where(df3['dayfin'] == 0, 1, df3['dayfin'])
    #set dates
    from datetime import datetime
    fd = []
    for i in range(0,df3.shape[0]):
        nd= datetime(df3['yearfin'][i],df3['monthfin'][i],df3['dayfin'][i])
        fd.append(nd)
    df3['findate'] = fd
    #keep only final date
    df4 = df3['findate'].sort_values(ascending=True)
    #sort by dates
    #df4 = df4.sort_values(ascending=1)
    #reset index to coursera reporting
    df4 = df4.reset_index()
    #final output
    fs = df4['index']
    
    return fs